In [ ]:
import copy
import numpy as np
import pyproj as pj


def get_xy(lat, lon):
    
    # setup your projections
    crs_wgs = pj.Proj(init='epsg:4326') # assuming you're using WGS84 geographic
    crs_bng = pj.Proj(init='epsg:27700') # use a locally appropriate projected CRS

    # then cast your geographic coordinate pair to the projected system
    x, y = pj.transform(crs_wgs, crs_bng, lon, lat)
    
    return x, y

def normalize(i):
    return i/np.linalg.norm(i)

file = open("final_project_point_cloud.fuse", "r")
txt_data = file.readlines()

raw_data = np.zeros((len(txt_data), len(txt_data[0].split())))
xyz_data = np.zeros((len(txt_data), len(txt_data[0].split())))

for idx, line in enumerate(txt_data):
    raw_data[idx,:] = line.split()
    xyz_data[idx,:] = copy.deepcopy(raw_data[idx,:])
    
x, y = get_xy(xyz_data[:, 0], xyz_data[:, 1])
xyz_data[:, 0] = np.array(x)
xyz_data[:, 1] = np.array(y)

print np.amax(xyz_data, axis=0)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt
plt.ion()
fig = plt.figure()
ax = Axes3D(fig)
num = xyz_data.shape[0]
skip=100
norm=mpl.colors.Normalize(vmin=0,vmax=255)
sc = ax.scatter(xyz_data[0:num:skip,0], xyz_data[0:num:skip,1], \
                xyz_data[0:num:skip,2], c=xyz_data[0:num:skip,3], norm=norm)
plt.colorbar(sc)
plt.show()

In [ ]:
#filter out low intensity points
line = xyz_data[np.logical_and(xyz_data[:,3]>80, xyz_data[:,3]<85)]
num = line.shape[0]
skip=1
fig1 = plt.figure()
ax1 = Axes3D(fig1)
sc1 = ax1.scatter(line[0:num:skip,0], line[0:num:skip,1], \
                 line[0:num:skip,2], c=line[0:num:skip,3], norm=norm)
plt.colorbar(sc1)
plt.show()

In [ ]:
# 2d clustering based on line structure in x-y plane first
fig2=plt.figure()
ax2 = fig2.add_subplot(111)
xy = line[:, 0:2]
xy = xy - np.min(xy, axis=0)
ax2.scatter(xy[:,0], xy[:,1])
plt.show()
# initialization of m and c
A = np.vstack([xy[:,0], np.ones(len(xy))]).T
m, c = np.linalg.lstsq(A, xy[:,1])[0]
c1 = c-20# + np.random.normal(0, 10)
c2 = c + np.random.normal(0, 10)
c3 = c+20# + np.random.normal(0, 10)
plt.plot(xy[:,0], m*xy[:,0] + c1, 'r')
plt.plot(xy[:,0], m*xy[:,0] + c2, 'g')
plt.plot(xy[:,0], m*xy[:,0] + c3, 'b')

for i in range(100):
    # classify
    y1 = np.absolute(A.dot([[m],[c1]]) - xy[:,[1]])
    y2 = np.absolute(A.dot([[m],[c2]]) - xy[:,[1]])
    y3 = np.absolute(A.dot([[m],[c3]]) - xy[:,[1]])
    k1 = np.squeeze(np.logical_and(np.logical_and(y1<y2, y1<y3),y1<10))
    k2 = np.squeeze(np.logical_and(np.logical_and(y2<y1, y2<y3),y2<10))
    k3 = np.squeeze(np.logical_and(np.logical_and(y3<y1, y3<y2),y3<10))
    # update
    A1 = np.vstack([xy[k1,0], np.ones(len(xy[k1]))]).T
    A2 = np.vstack([xy[k2,0], np.ones(len(xy[k2]))]).T
    A3 = np.vstack([xy[k3,0], np.ones(len(xy[k3]))]).T
    m1, c1 = np.linalg.lstsq(A1, xy[k1,1])[0]
    m2, c2 = np.linalg.lstsq(A2, xy[k2,1])[0]
    m3, c3 = np.linalg.lstsq(A3, xy[k3,1])[0]
    m = (m1+m2+m3)/3.0
    # replot
    ax2.clear()
    plt.scatter(xy[:,0], xy[:,1])
    plt.plot(xy[:,0], m*xy[:,0] + c1, 'r')
    plt.plot(xy[:,0], m*xy[:,0] + c2, 'g')
    plt.plot(xy[:,0], m*xy[:,0] + c3, 'b')


In [ ]:
# 3d polyline fitting based on 2d clusters
fig3=plt.figure()
ax3 = Axes3D(fig3)
xyz = line[:, 0:3]
xyz = xyz - np.min(xyz, axis=0)
ax3.scatter(xyz[:,0], xyz[:,1], xyz[:,2])
# divide into K clusters
K1 = xyz[k1,:]
K2 = xyz[k2,:]
K3 = xyz[k3,:]
K=[K1,K2,K3]
# polyline fitting
from numpy.polynomial import polynomial as P
xs=np.arange(100)
diff = np.min(xyz,axis=0)
for i in range(len(K)):
    X = K[i][:,0]
    Y = K[i][:,1:3]
    p=P.polyfit(X,Y,2)
    ax3.plot(xs, p[0,0] + p[1,0]*xs + p[2,0]*xs**2, \
             p[0,1] + p[1,1]*xs + p[2,1]*xs**2)
    ax.plot(xs+diff[0], p[0,0] + p[1,0]*xs + p[2,0]*xs**2 +diff[1], \
             p[0,1] + p[1,1]*xs + p[2,1]*xs**2 +diff[2], 'r')
    #p1=P.polyfit(X,Y,1)
    #ax3.plot(xs, p1[0,0] + p1[1,0]*xs, p1[0,1] + p1[1,1]*xs, 'r')

In [ ]:
# # failed approach - by 3d clustering based on 2d clustering
# fig3=plt.figure()
# ax3 = Axes3D(fig3)
# xyz = line[:, 0:3]
# xyz = xyz - np.min(xyz, axis=0)
# ax3.scatter(xyz[:,0], xyz[:,1], xyz[:,2])
# # initialization of a, b, d
# xtemp = np.arange(100)
# ytemp1 = m*xtemp+c1
# ytemp2 = m*xtemp+c2
# ytemp3 = m*xtemp+c3
# A1 = np.vstack([xtemp, ytemp1, np.ones(100)]).T
# A2 = np.vstack([xtemp, ytemp2, np.ones(100)]).T
# A3 = np.vstack([xtemp, ytemp3, np.ones(100)]).T
# a1, b1, d1 = np.linalg.lstsq(A1, np.zeros((100,1)))[0]
# a2, b2, d2 = np.linalg.lstsq(A2, np.zeros((100,1)))[0]
# a3, b3, d3 = np.linalg.lstsq(A3, np.zeros((100,1)))[0]
# plt.plot(xtemp,ytemp1, a1*xtemp+b1*ytemp1+d1, 'r')
# plt.plot(xtemp,ytemp2, a2*xtemp+b2*ytemp2+d2, 'g')
# plt.plot(xtemp,ytemp3, a3*xtemp+b3*ytemp3+d3, 'b')

# # improve on a, b, d
# for i in range(10):
#     # classify
#     A = np.vstack([xyz[:,0], xyz[:,1], np.ones(len(xyz))]).T
#     a1=[m]; a2=[m]; a3=[m]
#     b1=[-1];b2=[-1];b3=[-1]
#     d1=[c1];d2=[c2];d3=[c3]
#     y1 = np.absolute(A.dot(np.array([a1,b1,d1])) - xyz[:,[2]])
#     y2 = np.absolute(A.dot(np.array([a2,b2,d2])) - xyz[:,[2]])
#     y3 = np.absolute(A.dot(np.array([a3,b3,d3])) - xyz[:,[2]])
#     k1 = np.squeeze(np.logical_and(np.logical_and(y1<y2, y1<y3),y1<30))
#     k2 = np.squeeze(np.logical_and(np.logical_and(y2<y1, y2<y3),y2<30))
#     k3 = np.squeeze(np.logical_and(np.logical_and(y3<y1, y3<y2),y3<30))
#     # update
#     A1 = np.vstack([xyz[k1,0],xyz[k1,1],np.ones(len(xyz[k1]))]).T
#     A2 = np.vstack([xyz[k2,0],xyz[k2,1],np.ones(len(xyz[k2]))]).T
#     A3 = np.vstack([xyz[k3,0],xyz[k3,1],np.ones(len(xyz[k3]))]).T
#     a1,b1,d1 = np.linalg.lstsq(A1, 10*xyz[k1,2])[0]
#     a2,b2,d2 = np.linalg.lstsq(A2, 10*xyz[k2,2])[0]
#     a3,b3,d3 = np.linalg.lstsq(A3, 10*xyz[k3,2])[0]
#     # replot
#     ax3.clear()
#     plt.scatter(xyz[:,0], xyz[:,1], xyz[:,2])
#     plt.plot(xyz[k1,0],xyz[k1,1],a1*xyz[k1,0]+b1*xyz[k1,0]+d1,'r')
#     plt.plot(xyz[k2,0],xyz[k2,1],a2*xyz[k2,0]+b2*xyz[k2,0]+d2,'g')
#     plt.plot(xyz[k3,0],xyz[k3,1],a3*xyz[k3,0]+b3*xyz[k3,0]+d3,'b')
    

In [ ]:
# # failed approach - by histogram
# Xmat = np.zeros((num,num))
# Ymat = np.zeros((num,num))
# for i in range(num):
#     for j in range(i+1):
#         Xmat[i,j] = xy[i,0]-xy[j,0]
#         Ymat[i,j] = xy[i,1]-xy[j,1]
# Xmat = np.reshape(Xmat,(-1,1))
# Ymat = np.reshape(Ymat,(-1,1))
# Ymat = Ymat[Xmat[:,0]!=0]
# Xmat = Xmat[Xmat[:,0]!=0]
# slope = np.divide(Ymat, Xmat)
# slope = slope[np.logical_and(slope>-5, slope<10)]
# plt.figure()
# plt.scatter(np.arange(slope.shape[0]), slope)
# plt.show()

In [ ]:
# # failed approach - by hough transform
# from skimage.transform import(hough_line, hough_line_peaks, \
#                               probabilistic_hough_line)

# fig3, axes = plt.subplots(1,3,figsize=(15,6))
# ax = axes.ravel()
# ax[0].imshow(xy)

# h, theta, d = hough_line(xy)
# ax[1].imshow(np.log(1 + h),
#              extent=[np.rad2deg(theta[-1]), np.rad2deg(theta[0]), \
#                      d[-1], d[0]])
# ax[2].imshow(xy)
# for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
#     y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
#     y1 = (dist - xy.shape[1] * np.cos(angle)) / np.sin(angle)
#     ax[2].plot((0, xy.shape[1]), (y0, y1), '-r')

# plt.show()